In [1]:
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from collections import Counter

In [2]:
mdl = load_model('loan_model.h5')

In [3]:
df = pd.read_csv('bankloan.csv')
df = df.dropna()
df = df.drop('Loan_ID', axis=1)
df['LoanAmount'] = (df['LoanAmount']*1000).astype(int)
y_pre = df['Loan_Status']
X_pre = df.drop('Loan_Status', axis=1)
dm_X = pd.get_dummies(X_pre)
dm_y = y_pre.map(dict(Y=1, N=0))

In [4]:
X_test = dm_X.sample(20)
X_test.sample(2)

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
357,1.0,3875,0.0,67000,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1
288,0.0,4124,0.0,115000,360.0,1.0,1,0,1,0,1,0,1,0,0,1,0


In [5]:
sc=MinMaxScaler()
X_test = sc.fit_transform(X_test)

now X-test transformed to ndarray in act as input 

In [7]:
y_pred = mdl.predict(X_test)
y_pred = y_pred>0.58
y_pred # output

array([[ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False]])

In [8]:
val = pd.DataFrame(y_pred, columns=['Status'])
val = val.replace({True:'Approved', False:'Rejected'})
val

,Status
0,Approved
1,Approved
2,Rejected
3,Rejected
4,Rejected
5,Rejected
6,Rejected
7,Rejected
8,Approved
9,Approved


In [10]:
from imblearn.over_sampling import SMOTE
import joblib as jb

smote = SMOTE(sampling_strategy='auto')
X1, y = smote.fit_resample(dm_X,dm_y)
sc = MinMaxScaler()
X = sc.fit(X1)
filename = 'scalers.pkl'
jb.dump(X, filename)

['scalers.pkl']

In [12]:
X = sc.transform(X1)

In [16]:
type(sc.fit(X1))

sklearn.preprocessing._data.MinMaxScaler

In [11]:
X

MinMaxScaler()

In [13]:
X

array([[0.33333333, 0.05482993, 0.0445666 , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.03525046, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.03009276, 0.06968703, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.41444942, 0.05675943, 0.11551597, ..., 0.        , 0.        ,
        0.        ],
       [0.05525174, 0.05240569, 0.01814594, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.15946815, 0.01926372, ..., 0.        , 0.        ,
        0.        ]])

In [54]:
unit = {
    "Dependents": 2,
    "ApplicantIncome": 2500, 
    "CoapplicantIncome": 1840,
    "LoanAmount": 109,
    "Loan_Amount_Term": 360, 
    "Credit_History" : 1, 
    "Gender_Female": 0, 
    "Gender_Male": 1,
    "Married_No": 0, 
    "Married_Yes": 1, 
    "Education_Graduate": 1,
    "Education_Not Graduate": 0, 
    "Self_Employed_No": 1, 
    "Self_Employed_Yes": 0,
    "Property_Area_Rural": 0, 
    "Property_Area_Semiurban": 0,
    "Property_Area_Urban": 1
}

In [55]:
import numpy as np

In [56]:
n_unit = np.array(list(unit.values()))

In [57]:
n_unit

array([   2, 2500, 1840,  109,  360,    1,    0,    1,    0,    1,    1,
          0,    1,    0,    0,    0,    1])

In [58]:
n_unit = n_unit.reshape(1,-1)
n_unit

array([[   2, 2500, 1840,  109,  360,    1,    0,    1,    0,    1,    1,
           0,    1,    0,    0,    0,    1]])

In [59]:
X = sc.transform(n_unit)
X

array([[ 0.66666667,  0.02906617,  0.05437834, -0.01504399,  0.72972973,
         1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ]])

In [60]:
y_pred = mdl.predict(X)
y_pred

array([[0.82193655]], dtype=float32)